In [1]:
from pyomo.environ import *
import matplotlib.pyplot as plt
import numpy as np
import random 
import pandas as pd

In [2]:
df = pd.read_csv("paper.csv", delimiter=',', skiprows=0, low_memory=False)
df

,w,n
0,14,211
1,31,395
2,36,610
3,45,97


In [25]:
model = AbstractModel()
model.N =Param(mutable=True, initialize=455) 
model.i = RangeSet(1,model.N)
model.j = RangeSet(1,4)
model.U = Var(model.i, within=Binary, initialize=0)
model.x = Var(model.i, model.j, bounds=(0,5), domain=NonNegativeIntegers,initialize=0)
model.W =Param(model.j, mutable=True, initialize=1) 
model.number =Param(model.j, mutable=True, initialize=1) 
def rule_C1(model,i):
        return sum(model.W[j]*model.x[i,j] for j in model.j) <=100*model.U[i]
model.C1   = Constraint(model.i,rule=rule_C1)
def rule_C2(model,j):
        return sum(model.x[i,j] for i in model.i) ==model.number[j]
model.C2   = Constraint(model.j,rule=rule_C2)
def rule_OF(model):
    return sum(model.U[i] for i in model.i)
model.obj1 = Objective(rule=rule_OF, sense=minimize)

In [26]:
instance = model.create_instance()
for j in instance.j:
    instance.number[j]=df.iloc[j-1,1]
    instance.W[j]=df.iloc[j-1,0]

In [27]:
opt = SolverFactory('gurobi')
results=opt.solve(instance)
print('OF= ',value(instance.obj1))
from pyomo.opt import SolverStatus, TerminationCondition
if (results.solver.status == SolverStatus.ok) and (results.solver.termination_condition == TerminationCondition.optimal):
     print ("this is feasible and optimal")
elif results.solver.termination_condition == TerminationCondition.infeasible:
     print ("do something about it? or exit?")
else:
     print (str(results.solver))

OF=  453.0
this is feasible and optimal


In [18]:
print('   U   x ')
for i in instance.i:
    if value(instance.U[i])==1:
        print(i,sum(value(instance.W[j]*instance.x[i,j]) for j in instance.j) )  
        for j in instance.j:
            if value(instance.x[i,j])>0:
                print(i,j,value(instance.x[i,j]))
                
for j in instance.j:
    print(j, sum(value(instance.x[i,j]) for i in instance.i))

   U   x 
1 200.0
1 1 4.0
1 3 4.0
2 198.0
2 3 3.0
2 4 2.0
3 200.0
3 1 4.0
3 3 4.0
4 200.0
4 1 4.0
4 3 4.0
5 198.0
5 3 3.0
5 4 2.0
6 194.0
6 1 1.0
6 3 5.0
7 198.0
7 3 3.0
7 4 2.0
8 194.0
8 1 1.0
8 3 5.0
9 198.0
9 1 2.0
9 2 2.0
9 3 3.0
10 198.0
10 1 1.0
10 2 1.0
10 3 3.0
10 4 1.0
11 198.0
11 3 3.0
11 4 2.0
12 194.0
12 1 1.0
12 3 5.0
13 200.0
13 1 4.0
13 3 4.0
14 196.0
14 2 4.0
14 3 2.0
15 196.0
15 2 4.0
15 3 2.0
16 198.0
16 3 3.0
16 4 2.0
17 200.0
17 1 4.0
17 3 4.0
18 198.0
18 1 1.0
18 2 1.0
18 3 3.0
18 4 1.0
19 196.0
19 2 4.0
19 3 2.0
20 198.0
20 3 3.0
20 4 2.0
21 196.0
21 2 4.0
21 3 2.0
22 194.0
22 1 1.0
22 3 5.0
23 200.0
23 1 4.0
23 3 4.0
24 196.0
24 2 4.0
24 3 2.0
25 194.0
25 1 1.0
25 3 5.0
26 198.0
26 3 3.0
26 4 2.0
27 194.0
27 1 1.0
27 3 5.0
28 196.0
28 2 4.0
28 3 2.0
29 198.0
29 1 2.0
29 2 2.0
29 3 3.0
30 200.0
30 1 4.0
30 3 4.0
31 196.0
31 2 4.0
31 3 2.0
32 196.0
32 2 4.0
32 3 2.0
33 200.0
33 1 4.0
33 3 4.0
34 198.0
34 3 3.0
34 4 2.0
35 196.0
35 2 4.0
35 3 2.0
36 196.0
36 2 4.0
3